In [4]:
# !pip install dimod
# !pip install pyscf
# !pip install openfermion
# !pip install symengine
import math

from openfermionpyscf import run_pyscf
from openfermion.transforms import binary_code_transform, bravyi_kitaev_code, get_fermion_operator
# from openfermion.hamiltonians import MolecularData
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.utils import count_qubits
from pyscf import gto, scf, mcscf

from helper_functions import *
from XBK_method import *
from QCC_method import *

from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_ground_state, get_sparse_operator
import numpy
import scipy
import scipy.linalg

In [19]:
#create molecule
name = 'H3'
charge = 1
multiplicity = 1
basis = 'sto-6g'

bond_length = 1.1
geometry = get_molGeometry(name, bond_length)
    
molecule = MolecularData(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge
)

In [20]:
#run RHF calculations
driver = run_pyscf(molecule, run_scf=True)
hf_energy = float(molecule.hf_energy)
hf_data = molecule._pyscf_data['scf']

# print(molecule.canonical_orbitals)
print(hf_energy)

-1.2400693045269722


In [21]:
#define active space
n_active_electrons = 2
n_active_orbitals = 3
occupied_indices, active_indices = get_active_space(molecule, n_active_electrons, n_active_orbitals)

#run CASCI calculations
casci_data = hf_data.CASCI(n_active_orbitals, n_active_electrons).run(verbose=False)
casci_energy = float(casci_data.e_tot)

print(casci_energy)

-1.2722647705358239


In [22]:
#convert to fermionic Hamiltonian
molecular_H = molecule.get_molecular_hamiltonian(occupied_indices=occupied_indices, active_indices=active_indices)
if molecular_H[()] == None:
    molecular_H[()] = 0
fermionic_H = get_fermion_operator(molecular_H)
# print(fermionic_H)
#add penalty term to ensure correct number of electrons in ground state
weight = 5
penalty_term = FermionOperator('', n_active_electrons)

for i in range(molecular_H.n_qubits):
    penalty_term += FermionOperator(str(i)+'^ '+str(i), -1)
fermionic_H += weight*penalty_term**2

print(fermionic_H)

21.443210575236364 [] +
-21.623075395384372 [0^ 0] +
5.0 [0^ 0 0^ 0] +
5.0 [0^ 0 1^ 1] +
5.0 [0^ 0 2^ 2] +
5.0 [0^ 0 3^ 3] +
5.0 [0^ 0 4^ 4] +
5.0 [0^ 0 5^ 5] +
0.28143545550270405 [0^ 0^ 0 0] +
0.07258994740907981 [0^ 0^ 2 2] +
0.07258994740907987 [0^ 0^ 4 4] +
0.28143545550270405 [0^ 1^ 1 0] +
0.07258994740907981 [0^ 1^ 3 2] +
0.07258994740907987 [0^ 1^ 5 4] +
0.07258994740907981 [0^ 2^ 0 2] +
0.27480121211888675 [0^ 2^ 2 0] +
0.04853533511404069 [0^ 2^ 2 2] +
-0.011405586796407748 [0^ 2^ 2 4] +
-0.011405586796407779 [0^ 2^ 4 2] +
-0.048535335114040595 [0^ 2^ 4 4] +
0.07258994740907981 [0^ 3^ 1 2] +
0.27480121211888675 [0^ 3^ 3 0] +
0.04853533511404069 [0^ 3^ 3 2] +
-0.011405586796407748 [0^ 3^ 3 4] +
-0.011405586796407779 [0^ 3^ 5 2] +
-0.048535335114040595 [0^ 3^ 5 4] +
0.07258994740907987 [0^ 4^ 0 4] +
-0.011405586796407779 [0^ 4^ 2 2] +
-0.048535335114040595 [0^ 4^ 2 4] +
0.27480121211888686 [0^ 4^ 4 0] +
-0.048535335114040595 [0^ 4^ 4 2] +
0.011405586796407944 [0^ 4^ 4 4] +
0.07

In [23]:
#convert to Pauli operator Hamiltonian
binary_code = bravyi_kitaev_code(molecular_H.n_qubits)
qubit_H = binary_code_transform(fermionic_H, binary_code)
qubit_H.compress()

#apply symmetry reductions and calculate minimum eigenvalue (should be equal to CASCI energy)
sectors = taper_qubits(qubit_H)
qubit_H, min_eigenvalue = sector_with_ground(sectors)
m = count_qubits(qubit_H)

print(min_eigenvalue, '\n')
print(qubit_H)

-1.27226477053585 

12.052315964211402 [] +
-0.024267667557020367 [X0] +
0.024267667557020346 [X0 X1 X2] +
-0.005702793398203908 [X0 X1 Z2 X3] +
0.02426766755702037 [X0 X1 Z2 Z3] +
-0.005702793398203908 [X0 X1 X3] +
0.02426766755702037 [X0 X1 Z3] +
0.024267667557020346 [X0 Y1 Y2] +
0.005702793398203964 [X0 Z1 X2 X3] +
-0.024267667557020346 [X0 Z1 X2 Z3] +
0.02426766755702034 [X0 Z1 Z2] +
-0.005702793398203964 [X0 Z1 X3] +
0.02426766755702034 [X0 Z1 Z3] +
0.005702793398203964 [X0 X2 X3] +
-0.024267667557020346 [X0 X2 Z3] +
0.005702793398203874 [X0 Z2 X3] +
-0.024267667557020367 [X0 Z2 Z3] +
-0.005702793398203908 [Y0 X1 X2 Y3] +
-0.005702793398203908 [Y0 X1 Y2 X3] +
0.024267667557020346 [Y0 X1 Y2 Z3] +
0.005702793398203908 [Y0 Y1 X2 X3] +
-0.024267667557020346 [Y0 Y1 X2 Z3] +
-0.005702793398203908 [Y0 Y1 Y2 Y3] +
-0.005702793398203908 [Y0 Y1 Z2 X3] +
0.02426766755702034 [Y0 Y1 Z2 Z3] +
-0.005702793398203908 [Y0 Y1 X3] +
0.02426766755702034 [Y0 Y1 Z3] +
0.005702793398203874 [Y0 Z1 Y2 X3] 

In [26]:
#set sampler to perform the annealing
# !pip install dwave-ocean-sdk
from neal import SimulatedAnnealingSampler
sampler = SimulatedAnnealingSampler() #uses simulated annealing, see D-Wave's ocean sdk for more options

In [27]:
### XBK method ###

#set r value
r = 4

#construct qubit Hamiltonians and C terms for XBK method
qubit_Hs, qubit_Cs = [],[]
for p in range(int(math.ceil(r/2+1))):
    qubit_Hs += [XBK_transform(qubit_H, r, p)]
    qubit_Cs += [construct_C(m, r, p)]

#run XBK method
XBK_energy, ground_state = XBK(qubit_Hs, qubit_Cs, r, sampler, starting_lam=0, num_samples=1000, strength=1e3, verbose=False)

print(XBK_energy)
print(ground_state) #ground state in rm-qubit space

-1.24006930443866
[1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [28]:
### QCC method ###

#set number of Bloch angle and entangler amplitude foldings
angle_folds = 2
amplitude_folds = 1

#create dictionary of QubitOperator entanglers
entanglers = {'IYZI': QubitOperator('Y1 Z2'), 'IZYI': QubitOperator('Z1 Y2'),
              'IXYI': QubitOperator('X1 Y2'), 'IYXI': QubitOperator('Y1 X2')}

#run QCC method
QCC_energy, variables = QCC(qubit_H, entanglers, angle_folds, amplitude_folds, sampler, num_cycles=10, num_samples=1000, strength=1e3)
    
print(QCC_energy)
print(variables)

-0.8367203530506231
{phi0: 5.623308632608373, phi1: 6.283185307179586, phi2: 3.141592653589793, phi3: 2.8431514821105015, the0: 3.141592653589793, the1: 1.2581817802130424, the2: 1.2394726282922657, the3: 1.7007244472453488e-08, tau0: 3.2418058608575913, tau1: 3.1416272184587113, tau2: 4.396053380596374, tau3: 5.986532147902271}


EXPERIMENTS

In [70]:
import pyscf
mol = pyscf.M(
    atom = 'H 0 0 0; F 0 0 1.1',  # in Angstrom
    basis = '6-31g',
    symmetry = True,
)
# print(mol.atom)
myhf = mol.RHF().run()
# print(myhf.get_occ())
#
# create an FCI solver based on the SCF object
#
# cisolver = pyscf.fci.FCI(myhf)
# print('E(FCI) = %.12f' % cisolver.kernel()[0])

converged SCF energy = -99.9593211672818


In [73]:
from pyscf import gto, scf, ao2mo

'''
A simple example to call integral transformation for given orbitals
'''

mol = gto.Mole()
mol.build(
    atom = 'H 0 0 0; F 0 0 1.1',  # in Angstrom
    basis = 'ccpvdz',
    symmetry = True,
)

myhf = scf.RHF(mol)
myhf.kernel()

orb = myhf.mo_coeff
eri_4fold = ao2mo.kernel(mol, orb)
print('MO integrals (ij|kl) with 4-fold symmetry i>=j, k>=l have shape %s' %
      str(eri_4fold.shape))


#
# Starting from PySCF-1.7, the MO integrals can be computed with the code
# below.
#
import pyscf
mol = pyscf.M(
    atom = 'H 0 0 0; F 0 0 1.1',  # in Angstrom
    basis = 'ccpvdz',
    symmetry = True,
)
orb = mol.RHF().run().mo_coeff
eri_4fold = mol.ao2mo(orb)
print(eri_4fold[0])

converged SCF energy = -99.9873974403488
MO integrals (ij|kl) with 4-fold symmetry i>=j, k>=l have shape (190, 190)
converged SCF energy = -99.9873974403488
[ 5.35675278 -0.52831321  1.25358247 -0.09750893  0.09618362  1.02774801
  0.          0.          0.          1.22238529  0.          0.
  0.          0.          1.22238529  0.11576072 -0.17902118  0.27038076
  0.          0.          0.496825    0.04357091 -0.06704457  0.23163625
  0.          0.          0.20430308  0.52078992  0.          0.
  0.          0.46777722  0.          0.          0.          0.83934381
  0.          0.          0.          0.          0.46777722  0.
  0.          0.          0.83934381  0.18301991 -0.17064103 -0.40562827
  0.          0.         -0.1299511  -0.14695615  0.          0.
  0.90763646  0.          0.          0.         -0.35247869  0.
  0.          0.         -0.31506522  0.          0.          0.70668745
  0.          0.          0.          0.         -0.35247869  0.
  0.          0